In [1]:
import numpy as np
import pandas as pd
import os
import copy

# Settings
PATH = './'  # Current directory where your CSVs are located
OUTPUT_FILE = 'submission.csv'

def h_blend(params, _update={}):
    if 'subwts' in _update:
        params['subwts'] = _update['subwts']
    
    dk = copy.deepcopy(params)
    dk['asc'] = params['type_sort'][1]
    dk['desc'] = params['type_sort'][2]
    dk['id'] = params['id_target'][0]
    dk['target'] = params['id_target'][1]

    def read_file(dk, i):
        name = dk["subm"][i]["name"]
        return pd.read_csv(os.path.join(dk['path'], f"{name}.csv")).rename(columns={dk["target"]: name})
        
    def merge_submissions(dfs):
        df_merged = dfs[0]
        for i in range(1, len(dfs)):
            df_merged = pd.merge(df_merged, dfs[i], on=[dk['id']])
        return df_merged

    def perform_blending(dk, direction):
        dfs = [read_file(dk, i) for i in range(len(dk["subm"]))]
        df_subms = merge_submissions(dfs)
        cols = [col for col in df_subms.columns if col != dk['id']]
        
        # Determine the rank of each model for each row
        def get_sorted_cols(x, reverse=(direction == 'desc')):
            vals = {c: x[c] for c in cols}.items()
            return [t[0] for t in sorted(vals, key=lambda k: k[1], reverse=reverse)]

        df_subms['alls'] = df_subms.apply(lambda x: get_sorted_cols(x), axis=1)
        
        weights = [subm['weight'] for subm in dk["subm"]]
        sub_weights = dk["subwts"]

        def calculate_score(x):
            # Weigh each model by its fixed weight PLUS a weight based on its rank (alls)
            indices = [x['alls'].index(c) for c in cols]
            return sum([x[cols[j]] * (weights[j] + sub_weights[indices[j]]) for j in range(len(cols))])

        df_subms[dk["target"]] = df_subms.apply(calculate_score, axis=1)
        return df_subms[[dk['id'], dk['target']]]
   
    # Blend Ascending and Descending sorted ensembles
    df_desc = perform_blending(dk, 'desc')
    df_asc = perform_blending(dk, 'asc')
    
    combined_scores = dk['desc'] * df_desc[dk['target']] + dk['asc'] * df_asc[dk['target']]
    return pd.DataFrame({dk['id']: df_desc[dk['id']], dk['target']: combined_scores})

def blend_aux(df_main, weights, df_aux, name):
    # Standard submission format loading
    sub = pd.read_csv('sample_submission.csv')
    sub['exam_score'] = df_aux['exam_score'] * weights[0] + df_main['exam_score'] * weights[1]
    sub.to_csv(f"{name}.csv", index=False)
    return sub

def process_aux_params(df_main, weights, dfs_aux, params_aux):
    for i in range(len(dfs_aux)):
        blend_aux(df_main, weights, dfs_aux[i], params_aux['subm'][i]['name'])
    return copy.deepcopy(params_aux)

# --- Configuration ---
params_Main = {
    'path': PATH,
    'id_target': ['id', "exam_score"],          
    'type_sort': ['asc/desc', 0.30, 0.70],
    'subm': [
        {'name': '8.54465', 'weight': 0.31},
        {'name': '8.54633', 'weight': 0.33},
        {'name': '8.54610', 'weight': 0.17},
        {'name': '8.54822', 'weight': 0.19},
    ]
}

params_Aux = {
    'path': PATH,
    'id_target': ['id', "exam_score"],          
    'type_sort': ['asc/desc', 0.30, 0.70],
    'subwts': [-0.25, 0.00, 0.55, -0.30],
    'subm': [
        {'name': 'Main+24', 'weight': 0.21},
        {'name': 'Main+25', 'weight': 0.08},
        {'name': 'Main+28', 'weight': 0.23},
        {'name': 'Main+29', 'weight': 0.48},
    ]
}

# Constants for Trend adjustment
ct1, ct2 = 1.00118, 1.00118
weights1, weights2, weights3 = [0.96, 0.04], [0.89, 0.11], [0.82, 0.18]

# Load specific anchor files
df24 = pd.read_csv(os.path.join(PATH, '8.54466.ct.csv'))
df25 = pd.read_csv(os.path.join(PATH, '8.54476.csv'))
df28 = pd.read_csv(os.path.join(PATH, '8.54465.ct.csv'))
df29 = pd.read_csv(os.path.join(PATH, '8.54462.csv')) # Ensure this file is in the folder
dfs_Aux = [df24, df25, df28, df29]

# --- Execute Triple Blend ---
print("Running Triple Blend and Trend Analysis...")
m1 = h_blend(params_Main, _update={'subwts': [0.55, -0.10, -0.20, -0.25]})
df1 = h_blend(process_aux_params(m1, weights1, dfs_Aux, params_Aux))

m2 = h_blend(params_Main, _update={'subwts': [0.11, -0.01, -0.03, -0.07]})
df2 = h_blend(process_aux_params(m2, weights2, dfs_Aux, params_Aux))

m3 = h_blend(params_Main, _update={'subwts': [0.55, -0.10, -0.20, -0.25]})
df3 = h_blend(process_aux_params(m3, weights3, dfs_Aux, params_Aux))

# --- Trend Correction Logic ---
df1.rename(columns={'exam_score': 'es1'}, inplace=True)
df2.rename(columns={'exam_score': 'es2'}, inplace=True)
df3.rename(columns={'exam_score': 'es3'}, inplace=True)
df_final = df1.merge(df2, on='id').merge(df3, on='id')

def calculate_trend(x):
    e1, e2, e3 = x['es1'], x['es2'], x['es3']
    if e1 < e3 and e2 < e3: 
        return e3 * (ct1 - 0.0001 * (e3 - e1))
    if e1 > e2 and e2 > e3: 
        return e3 / (ct2 - 0.0001 * (e1 - e3))
    return e3

df_final['exam_score'] = df_final.apply(calculate_trend, axis=1)

# Cleanup and Save
df_final[['id', 'exam_score']].to_csv(OUTPUT_FILE, index=False)
print(f"Final file '{OUTPUT_FILE}' generated. Submit this to get your target score.")

Running Triple Blend and Trend Analysis...
Final file 'submission.csv' generated. Submit this to get your target score.
